# Directional Stiffness plot of a Gyroid

In [1]:
from fedoo import *
import numpy as np
import time
from simcoon import simmit as sim
import matplotlib.pyplot as plt
from matplotlib import cm, colors
import os


## Plot the Mesh

In [8]:
from sidecar import Sidecar
from ipygany import Scene, PolyMesh

OUT = Util.ExportData(meshID)
OUT.toVTK("3D_Periodic_BC.vtk")

mesh_to_plot = PolyMesh.from_vtk('3D_Periodic_BC.vtk')
mesh_to_plot.default_color = 'gray'

sc = Sidecar(title='ipygany_gyroid')
scene = Scene([mesh])

with sc:
    display(scene)

## Generate the effective Stiffness Matrice

In [2]:
#--------------- Pre-Treatment --------------------------------------------------------
Util.ProblemDimension("3D")

Mesh.ImportFromFile('Mesh0.3.msh', meshID = "Domain")
meshID = "Domain"
mesh = Mesh.GetAll()['Domain']
crd = mesh.GetNodeCoordinates()
elm = mesh.GetElementTable()

#we define the type of Constitutive Law to be utilized
umat_name = 'ELISO'
props = np.array([[1e5, 0.3, 1]])
nstatev = 1

#Here we use Simcoon to determine the elastic stiffness matrix
L = sim.L_iso(1e5, 0.3, 'Enu')
props_test = sim.L_iso_props(L)
print('props', props_test)

#Weak Form
Material = ConstitutiveLaw.ElasticAnisotropic(L, ID = 'ElasticLaw')
wf = WeakForm.InternalForce("ElasticLaw", ID = "WeakForm", nlgeom=False)
# Assembly
assemb = Assembly.Create("WeakForm", "Domain", mesh.GetElementShape(), ID="Assembly")

L_eff = Homogen.GetHomogenizedStiffness(assemb, meshperio=False)

np.set_printoptions(precision=3, suppress=True)
print('L_eff = ', L_eff)

Mesh imported: "Domain" with elements tet4
props [1.e+05 3.e-01]
28419
28419
28419
28420
28420
28420
L_eff =  [[17158.638  7392.35   7401.052    20.507    25.706    12.588]
 [ 7392.35  17159.938  7403.182    19.967     7.78     24.052]
 [ 7401.052  7403.182 17166.323     8.491    23.431    22.291]
 [   20.507    19.967     8.491  5584.691     3.882     2.431]
 [   25.706     7.78     23.431     3.882  5584.403     1.389]
 [   12.588    24.052    22.291     2.431     1.389  5583.699]]


## Generate the directional Stiffness value

In [9]:
plt.rcParams['text.usetex'] = True
plt.rcParams["figure.figsize"] = (20,8)

phi = np.linspace(0,2*np.pi, 128) # the angle of the projection in the xy-plane
theta = np.linspace(0, np.pi, 128).reshape(128,1) # the angle from the polar axis, ie the polar angle

n_1 = np.sin(theta)*np.cos(phi)
n_2 = np.sin(theta)*np.sin(phi)
n_3 = np.cos(theta)*np.ones(128)

n = np.array([n_1*n_1, n_2*n_2, n_3*n_3, n_1*n_2, n_1*n_3, n_2*n_3]).transpose(1,2,0).reshape(128,128,1,6)

M = np.linalg.inv(L_eff)

S = (n@M@n.reshape(128,128,6,1)).reshape(128,128)

E = (1./S)
x = E*n_1
y = E*n_2
z = E*n_3

#E = E/E.max()

fig = plt.figure(figsize=plt.figaspect(1))  # Square figure
ax = fig.add_subplot(111, projection='3d')

# make the panes transparent
ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
# make the grid lines transparent
ax.xaxis._axinfo["grid"]['color'] =  (1,1,1,0)
ax.yaxis._axinfo["grid"]['color'] =  (1,1,1,0)
ax.zaxis._axinfo["grid"]['color'] =  (1,1,1,0)
ax.set_axis_off()

#ax.plot_surface(x, y, z, cmap='hot',c=E)

#norm = colors.Normalize(vmin = 0., vmax = 10000, clip = False)
Emin = np.min(E)
Eavg = np.average(E)
Emax = np.max(E)
norm = colors.Normalize(vmin = Emin, vmax = Emax, clip = False)
surf = ax.plot_surface(x, y, z, rstride=1, cstride=1, norm=norm, facecolors=cm.cividis(norm(E)),linewidth=0, antialiased=False, shade=False)

#ax.set_xlim(0,20000)
#ax.set_ylim(0,20000)
#ax.set_zlim(0,20000)
#ax.set_xlabel(r'$E_x$ (MPa)')
#ax.set_ylabel(r'$E_y$ (MPa)')
#ax.set_zlabel(r'$E_z$ (MPa)')

scalarmap = cm.ScalarMappable(cmap=plt.cm.cividis, norm=norm)
scalarmap.set_clim(np.min(E),np.max(E))
#m.set_array([])
cbar = plt.colorbar(scalarmap, orientation="horizontal", fraction=0.06, pad=-0.1, ticks=[Emin, Eavg, Emax])
cbar.ax.tick_params(labelsize='large')
cbar.set_label(r'directional stiffness $E$ (MPa)', size=15, labelpad=20)

#ax.figure.axes[0].tick_params(axis="both", labelsize=5)
ax.figure.axes[1].tick_params(axis="x", labelsize=20)

ax.azim = 30
ax.elev = 30

#Volume_mesh = Assembly.GetAll()['Assembling'].IntegrateField(np.ones_like(TensorStress[0]))

#plt.savefig("directional.png", transparent=True)

sc = Sidecar(title='directional')

with sc:
    display(plt.show())